In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from functools import reduce
import math
from collections import Counter
import csv
from datetime import datetime
import re
import copy
import json
from datetime import timedelta
from operator import itemgetter
%matplotlib inline

In [2]:
def now():
    return str(datetime.now().time())[:8]
def pr(strToPrint):
    print(now() + ' '+ strToPrint)

from IPython.display import Audio
sound_file = 'beep.wav'
# pd.set_option('display.max_colwidth', -1)

## Importing data

Importing a sample of the dataset:

In [3]:
pickle_filename = os.path.join('data','head_100k_pickle.pkl')
tw = pd.read_pickle(pickle_filename)

Importing the whole dataset

In [4]:
columns_header = ['id', 'userId', 'createdAt', 'text', 'longitude', 'latitude', 'placeId',
                  'inReplyTo', 'source', 'truncated', 'placeLatitude', 'placeLongitude', 'sourceName', 'sourceUrl',
                 'userName', 'screenName', 'followersCount', 'friendsCount', 'statusesCount',
                 'userLocation']

filename = os.path.join('data','twex.tsv') # 'sample.tsv')
pr('Starting to read file... (3 min)')
# tw = pd.read_csv(filename, sep='\t', encoding='utf-8', escapechar='\\', names=columns_header,
#                       quoting=csv.QUOTE_NONE, na_values='N', header=None)

pr('File is loaded!')
# Audio(url=sound_file, autoplay=True)

00:45:44 Starting to read file... (3 min)
00:45:44 File is loaded!


In [5]:
tw.head(2)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
0,9514097914,1.7341e+07,2010-02-23 05:55:51,Guuuuten Morgen! :-),7.43926,46.9489,NaN,NaN,197,NaN,NaN,NaN,TwitBird,http://www.nibirutech.com,Tilman Jentzsch,blickwechsel,586,508.0,9016.0,"Bern, Switzerland"
1,9514846412,7.19828e+06,2010-02-23 06:22:40,Still the best coffee in town — at La Stanza h...,8.53781,47.3678,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Nico Luchsinger,halbluchs,1820,703.0,4687.0,"Zurich, Switzerland"


## Extrating hashtags

In [6]:
def extract_hashtags(text):
    ht_list = re.findall(r"#(\w+)", text)
    non_empty_hts = list(filter((lambda ht: ht != []), ht_list))
    lowerCharList = [ht.lower() for ht in non_empty_hts]
    return lowerCharList

In [7]:
pr('Making hashtags... (2 min)')
tw['hashtag'] = np.nan
tw.hashtag = tw.text.apply(lambda x: extract_hashtags(str(x)))
twh = tw.ix[tw.hashtag.apply(lambda x: len(x) != 0)]
pr('Done.')

00:45:45 Making hashtags... (2 min)
00:45:45 Done.


In [8]:
twh.head(2)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,...,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
8,9519737890,1.46579e+07,2010-02-23 09:59:41,"Magic spells run off after midnight, I guess s...",6.13870,46.175,NaN,NaN,1,NaN,...,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Javier Belmonte,vichango,167,277.0,2885.0,"Geneva, Switzerland",[fb]
10,9521789689,9.96202e+06,2010-02-23 11:28:27,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550,NaN,...,NaN,Gowalla,http://gowalla.com/,Thomas Winter,thwinter,1070,1359.0,3349.0,Hettlingen CH / SanJose Ca,[yam]


## Cleaning data and making date index

In [9]:
tw1 = twh.dropna(axis=0, how='any', subset=['text', 'createdAt'])
print('The data have been reduced from {} tweets to {} tweets.'.format(len(twh), len(tw1)))

The data have been reduced from 19719 tweets to 19719 tweets.


In [10]:
pr('Removing bad dates...')
twhCleanDate = tw1[tw1['createdAt'].str.len() == 19]
pr('Finished.')

00:45:45 Removing bad dates...
00:45:45 Finished.


In [11]:
pr('Starting to examine dates...')
import warnings
warnings.filterwarnings('ignore')
datetime_serie = twhCleanDate['createdAt'].convert_objects(convert_dates='coerce')
dateNotConvertible = datetime_serie[pd.isnull(datetime_serie)]
warnings.filterwarnings('default')
pr('There are {} dates that cannot be transformed.'.format(len(dateNotConvertible)))

00:45:45 Starting to examine dates...
00:45:46 There are 0 dates that cannot be transformed.


In [12]:
pr('Starting copy...') # (to avoid transformation problems)
tw5 = twhCleanDate.copy()
pr('Converting to datetime...')
tw5['createdAt'] = pd.to_datetime(twhCleanDate['createdAt'])
pr('Setting up new indices...')
tw5.index = tw5['createdAt']
pr('Deleting old "createdAt" column...')
del tw5['createdAt']
pr('Done!')
tw5.head(2)

00:45:46 Starting copy...
00:45:46 Converting to datetime...
00:45:46 Setting up new indices...
00:45:46 Deleting old "createdAt" column...
00:45:46 Done!


,id,userId,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
createdAt,,,,,,,,,,,,,,,,,,,,
2010-02-23 09:59:41,9519737890,1.46579e+07,"Magic spells run off after midnight, I guess s...",6.13870,46.175,NaN,NaN,1,NaN,NaN,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Javier Belmonte,vichango,167,277.0,2885.0,"Geneva, Switzerland",[fb]
2010-02-23 11:28:27,9521789689,9.96202e+06,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Thomas Winter,thwinter,1070,1359.0,3349.0,Hettlingen CH / SanJose Ca,[yam]


In [13]:
tw5['hashtag'][:6]

createdAt
2010-02-23 09:59:41               [fb]
2010-02-23 11:28:27              [yam]
2010-02-23 17:47:11          [24, vfb]
2010-02-23 18:19:03    [iphoneography]
2010-02-23 18:31:46     [partnermonth]
2010-02-24 06:09:23      [insider, fb]
Name: hashtag, dtype: object

## Let's put one hashtag per row

We will make a dataframe with one row = one hashtag. This will be done by going through the dataframe, and making in parallel a list of rows (with 1 hashtag per row) that needs to be added to the old dataframe.

In [14]:
addedHashtagsRowsList = []
def multiplyHashtagRows(row, columns):
    '''
    Examine each row. If there are multiple hashtags, it will return the first one.
    (so the first one will replace the list of hashtags in the df). Then for all the next ones,
    it will make a copy of the row in the addedHashtagsRowsList, (in a dictionary format).
    So this dictionary can in the end be transformed in a DF and added to the original DF.
    (The speed is increased a lot by doing it this way!)
    '''
    htList = row.hashtag
    if len(htList) > 1:
        ## Making the dictionary
        addedHashtag = {}
        addedHashtag['createdAt'] = row.name #the df index
        for col in columns:
            addedHashtag[col] = row[col]
        ## Copying the dict for each hashtag
        i = 1
        while i < len(htList) :
            deepCopy = copy.deepcopy(addedHashtag)
            deepCopy['hashtag'] = htList[i]
            addedHashtagsRowsList.append(deepCopy)
            i+=1
    return htList[0] # return the first hashtag

In [15]:
addedHashtagsRowsList = []
tw5_1 = tw5.copy()
pr('Multiplying the hashtag rows... (around 10 min)')
tw5_1['hashtag'] = tw5.apply(multiplyHashtagRows, args=[tw5.columns,], axis=1)
pr('Finished! {} rows will be added to the dataframe!'.format(len(addedHashtagsRowsList)))

00:45:46 Multiplying the hashtag rows... (around 10 min)
00:45:50 Finished! 8887 rows will be added to the dataframe!


In [16]:
pr('Starting to make the new dataframe with additionnal rows..')
addedHashtagsDf = pd.DataFrame(addedHashtagsRowsList)
addedHashtagsDf.set_index(['createdAt'], inplace=True)
pr('Starting to append the two df... Old df size = {}'.format(len(tw5_1)))
tw6 = tw5_1.append(addedHashtagsDf)
pr('Done! New df size = {}'.format(len(tw6)))

00:45:50 Starting to make the new dataframe with additionnal rows..
00:45:50 Starting to append the two df... Old df size = 19719
00:45:50 Done! New df size = 28606


In [17]:
print('Example hahshtag:')
tw6[tw6['hashtag'] == addedHashtagsRowsList[0]['hashtag']].head(3)

Example hahshtag:


,followersCount,friendsCount,hashtag,id,inReplyTo,latitude,longitude,placeId,placeLatitude,placeLongitude,screenName,source,sourceName,sourceUrl,statusesCount,text,truncated,userId,userLocation,userName
createdAt,,,,,,,,,,,,,,,,,,,,
2011-07-24 16:03:13,1121,763.0,vfb,95162125584039936,9.501397e+16,NaN,NaN,e401fb8eb4e7595a,47.5356,9.14004,eLd0raDo,14,Tweetbot for iOS,http://tapbots.com/tweetbot,6735.0,"@mikstweed Wie lange ist es her, dass der #vfb...",NaN,921241,"Kreuzlingen, TG, Switzerland",Markus Tressl
2010-02-23 17:47:11,1121,763.0,vfb,9535390586,NaN,47.6463,9.16570,NaN,NaN,NaN,eLd0raDo,550,Gowalla,http://gowalla.com/,6735.0,"So, Feierabend. Jetzt #24 und später #VfB. — a...",NaN,921241,"Kreuzlingen, TG, Switzerland",Markus Tressl
2010-12-01 17:25:56,1317,830.0,vfb,10021795398684672,NaN,46.9499,7.47071,e38a1a641d02f8db,46.9543,7.39491,chm,1,Twitter for iPhone,http://twitter.com/#!/download/iphone,13552.0,"So, und jetzt Flachmann suchen und dann ab ins...",NaN,120433,"Bern, Switzerland",chm


In [18]:
tw6.hashtag.head()

createdAt
2010-02-23 09:59:41               fb
2010-02-23 11:28:27              yam
2010-02-23 17:47:11               24
2010-02-23 18:19:03    iphoneography
2010-02-23 18:31:46     partnermonth
Name: hashtag, dtype: object

## Grouping per hashtags per day

In [19]:
tw6.dropna(subset=['longitude'], inplace=True)
tw6.dropna(subset=['latitude'], inplace=True)

In [20]:
tw6.latitude = tw6.latitude.apply(float)
tw6.longitude = tw6.longitude.apply(float)

In [21]:
delimiter = '_$$$_'
str_join = lambda x: delimiter.join(x)

Function that applies to a dataframe will group each row by day and aggregate all its content:

In [22]:
def aggDate(df):
    groupedDf = df.groupby(df.index.map(lambda x: x.date)).agg({'text' : str_join,
                                                          'longitude' : np.median,
                                                          'latitude' : np.median,
                                                          'hashtag' : lambda x: x.iloc[0], ## the first occurence
                                                          'numberOfTweets' : 'count'})
    return groupedDf

In [23]:
pr('Making column number of tweets')
tw6['numberOfTweets'] = 1
pr('Starting group by hastag...')
gp = tw6.groupby('hashtag')
pr('Starting to put hashtag in dictionary... (around 50 min)')

count = 0
lengp = len(gp)
printingValue = int(lengp / 10)
dictionary = {}
for hashtag, df in gp:
    dictionary[hashtag] = aggDate(df)
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/lengp*100))
pr('Finished operations! Dictionary with {} different hashtags.'.format(len(dictionary)))

00:45:51 Making column number of tweets
00:45:51 Starting group by hastag...
00:45:51 Starting to put hashtag in dictionary... (around 50 min)
00:45:57 10%
00:46:03 20%
00:46:09 30%
00:46:15 40%
00:46:21 50%
00:46:27 60%
00:46:33 70%
00:46:39 80%
00:46:45 90%
00:46:51 100%
00:46:51 Finished operations! Dictionary with 9410 different hashtags.


In [24]:
# aggDate(gp.get_group('twibisg')).head(3)
# dictionary['twibisg'].head(3)
print('Dictionary with hashtags dataframes:')
dictionary[list(dictionary.keys())[5]].head(4)

Dictionary with hashtags dataframes:


,latitude,hashtag,numberOfTweets,text,longitude
2011-10-07,47.6419,hadoopworld,1,@fgdrf1976 A few engineers from @MeMoNewsAG wi...,9.18538


## Event detection (with elimination of recurrent events)

Parameters that define events:

In [25]:
## Parameters of an event:
MIN_TOT_NB_TWEETS = 20 ## The hashtag must have happened at least this number of times to be considered.
MIN_NB_DAYS_WITH_HASHTAGS = 3 ## The hashtags must appear at least this number of different days to be considered.
MIN_NB_TWEETS_DURING_EVENT = 4 ## To be considered an event, the hashtag must happen at least this nb of times during the day.
THRESHOLD_ANOMALY_FACTOR = 1.5 ## The occurence of a hashtag during a single day must be above the mean by this FACTOR
                             ## multiplied by the std to be considered as an anomaly.
MAX_DURATION_OF_EVENT = timedelta(days=30) ## The maximum number of days we consider an event can happen
MIN_DURATION_BEFORE_NEW_EVENT = timedelta(days=304) ## (= 10 months) The min time that should pass before an event can happen
                                                    ## again and still be considered as event (ie. Christmas is an event
                                                    ## each year)

Helper functions to detect recurrent events that should be removed:

In [26]:
def isSpecificEventListIllegal(detectedEventDateList):
    '''
    Return true if the list of dates contain illegal tupples of events, so if the event is recurrent
    which would mean it is not a real event.
    '''
    def datesAreIllegal(date1, date2, date3):
        '''
        Return true if the 3 dates are not to be considered as regular events.
        '''
        ## Return if the difference is too small to be considered as 2 different events
        def diffIsSmall(timeDiff):  
            return timeDiff < MAX_DURATION_OF_EVENT

        ## Return true if the difference is not big enough to be an annual event.
        def isDiffSuspect(timeDiff):
            return timeDiff < MIN_DURATION_BEFORE_NEW_EVENT   

        diff1 = abs(date1 - date2)
        diff2 = abs(date2 - date3)
        diff3 = abs(date3 - date1)

        ## The difference is too small, it must be the same event
        if diffIsSmall(diff1) or diffIsSmall(diff2) or diffIsSmall(diff3):
            return False

        ## If there are at least 2 out of 3 suspect difference, then the dates are illegal
        if isDiffSuspect(diff1):
            return isDiffSuspect(diff2) or isDiffSuspect(diff3)
        else:
            return isDiffSuspect(diff2) and isDiffSuspect(diff3)
    
    ## MAIN FUNCTION : ##
    # Go through the list of events and try all "triples" to see if there is any illegal triples. This is a quickly done
    # code to do that. Code complexity bellow is in O(k^3), with k being the size of the list. We will apply this function
    # to n list so we will have an overall complexity in O(n*k^3). We can consider however that each list will
    # be small so k can be considered as constant and therefore the overall complexity will be in O(n).
    for i in range(len(detectedEventDateList) - 2):
        for j in range(i, len(detectedEventDateList) - 1):
            for k in range(j, len(detectedEventDateList)):
                if datesAreIllegal(detectedEventDateList[i], detectedEventDateList[j], detectedEventDateList[k]):
                    return True
    return False

Main method to detect event according to all our criterias.

In [27]:
pr('Starting to compute {} dict items to detect event. (4 min)'.format(len(dictionary)))
nbOfEventDetected = 0
count = 0
printingValue = int(len(dictionary) / 10)
for [h,df] in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df['event'] = False
    if len(df) > MIN_NB_DAYS_WITH_HASHTAGS:
        if df['numberOfTweets'].sum() > MIN_TOT_NB_TWEETS:
            threshold = df['numberOfTweets'].mean() + THRESHOLD_ANOMALY_FACTOR * df['numberOfTweets'].std()
            df['event'] = df.numberOfTweets.apply(lambda x: x > threshold and x > MIN_NB_TWEETS_DURING_EVENT)
            
            ## Remove recurrent events:
            detectedEventDf = df[df['event']]
            if len(detectedEventDf) > 2 and isSpecificEventListIllegal(detectedEventDf.index):
                df['event'] = False
            nbOfEventDetected += len(df[df['event']])
pr('Finished! Number of events detected = {}'.format(nbOfEventDetected))

00:46:51 Starting to compute 9410 dict items to detect event. (4 min)
00:46:52 10%
00:46:52 20%
00:46:53 30%
00:46:53 40%
00:46:54 50%
00:46:54 60%
00:46:55 70%
00:46:55 80%
00:46:56 90%
00:46:56 100%
00:46:56 Finished! Number of events detected = 37


## Merging close events and grouping into single event dataframe

Here, we have a function that is going to merge events that are considered as too "close" to each other to be considered individually.

In [28]:
def mergeCloseEvents(rowsList):
    '''
    Take a list of dictionary, where each dictionary is a "row" of the event df, which contained detected events.
    It will process the list to detect event that are close and merge them together.
    Return : the processed list of event.
    '''
    
    def areCloseEvents(event1, event2):
        '''
        Return true is 2 events dates are defined as "close"
        '''
        return abs(event1['date'] - event2['date']) < MAX_DURATION_OF_EVENT
        
    def mergeCloseEventsSublist(closeEventList):
        '''
        This will be applied to each close event sublist. It will merge all events into one unique event.
        The event will consist of the total number of tweets, with the concatenation of the tweet texts and the mean
        of longitude/latitude. A meanDate will be defined as a ponderated mean between all dates.
        The final date will be the one that is in the closeEventList and is closest to this mean date.
        We did this to keep the meaning of the date if it had some, and not have some meaningless "mean-date".
        '''
        latitude = 0
        longitude = 0
        numberOfTweets = 0
        text = ""
        originalDate = closeEventList[0]['date']
        dateDiff = timedelta(days=0)
        first = True        
        for tweet in closeEventList:
            longitude += tweet['longitude']
            latitude += tweet['latitude'] 
            numberOfTweets += tweet['numberOfTweets']
            if first:
                text = tweet['text']
                first = False
            else:
                text += delimiter + tweet['text']
                dateDiff = dateDiff + (tweet['date'] - originalDate) * tweet['numberOfTweets']

        ## It is multiplied by 2 then soustracted to round correctly to the nearest day
        meanDate = originalDate + 2* dateDiff / numberOfTweets - dateDiff / numberOfTweets        
        latitude = latitude / len(closeEventList)
        longitude = longitude / len(closeEventList)
        
        ## We are going to detect the event the closest to the mean date
        minSelectedDate = closeEventList[0]['date']
        minDistance = abs(closeEventList[0]['date'] - meanDate)
        for tweet in closeEventList:
            if abs(tweet['date'] - meanDate) < minDistance:
                minSelectedDate = tweet['date']    
        
        return {'date': minSelectedDate, 'hashtag': closeEventList[0]['hashtag'], 'text': text,
                    'longitude': longitude, 'latitude':latitude, 'numberOfTweets': numberOfTweets, }
    
    ############ -----  MAIN METHOD  ----- ############
    
    ## If the list is big enough, go through the list and form an export list and merge elements that needs to.
    if len(rowsList) < 2:
        return rowsList
    else:
        firstLastPosOfItemsToMerge = []
        sortedRowsList = sorted(rowsList, key=itemgetter('date')) 
        exportedEventList = []
        ## This goes through the *sorted* list and add the pair of indices (first indice and last indice) where events 
        ## that should be merged appear.
        lastEventWasClose = False
        firstItem = -1
        for i in range(0, len(sortedRowsList)-1):
            if areCloseEvents(sortedRowsList[i], sortedRowsList[i+1]):
                if not lastEventWasClose: # So it is the first pairs of the sublist of close events in the whole list
                    firstItem = i
                    lastEventWasClose = True
            else:
                if lastEventWasClose: # So the list has just ended.
                    exportedEventList.append(mergeCloseEventsSublist(sortedRowsList[firstItem:i+1]))
                    lastEventWasClose = False
                else: # The element is by itself, let's append it
                    exportedEventList.append(sortedRowsList[i])  
        if lastEventWasClose: # If there were events to merge till the last elem of list
            exportedEventList.append(mergeCloseEventsSublist(sortedRowsList[firstItem:len(sortedRowsList)]))
        else:
            exportedEventList.append(sortedRowsList[len(sortedRowsList)-1])
    
    return exportedEventList

This function will be applied to each dataframe. If a row is detected as an event, it will be added to the locaRowsList which will be used to make a general dataframe of all the events.

In [29]:
localRowsList = []
def applyToMakeEventDf(row):
    if row.event:
        rowToAdd = {'date': row.name, 'hashtag': row.hashtag, 'text': row.text,
                    'longitude': row.longitude, 'latitude':row.latitude, 'numberOfTweets': row.numberOfTweets, }
        global localRowsList
        localRowsList.append(rowToAdd)

In [30]:
eventRowsList = []
localRowsList = []
count = 0
printingValue = int(len(dictionary) / 10)

pr('Starting to make event df with {} dataframes. (around 6 min)'.format(len(dictionary)))
for h, df in dictionary.items():
    global localRowsList
    localRowsList = []
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
        
    df.apply(applyToMakeEventDf, axis=1)
    mergedList = mergeCloseEvents(localRowsList) # merging close events
    eventRowsList += mergedList

pr('Making new dataframe.')
new_events = pd.DataFrame(eventRowsList)
new_events.set_index(['date'], inplace=True)
pr('Finished! Dataframe with {} rows'.format(len(new_events)))

00:46:57 Starting to make event df with 9410 dataframes. (around 6 min)
00:46:57 10%
00:46:58 20%
00:46:59 30%
00:47:00 40%
00:47:00 50%
00:47:01 60%
00:47:02 70%
00:47:03 80%
00:47:04 90%
00:47:04 100%
00:47:04 Making new dataframe.
00:47:04 Finished! Dataframe with 33 rows


In [42]:
print('Events dataframe:')
new_events.head(10)

Events dataframe:


,hashtag,latitude,longitude,numberOfTweets,text
date,,,,,
2010-05-26,ipad,47.315700,8.552750,5,"@shibby Vielleicht ""noch"" nicht? Das #iPad mit..."
2010-12-08,switzerland,47.363800,8.534930,9,India's cell-phone marketing: 545 million hand...
2010-07-03,twittboat,47.375600,8.542245,12,Jetzt höre ich schon NRJ um mich mental auf al...
2011-07-16,osx,47.499500,8.728720,5,"Wäre das schön, wenn's auf #OSX sowas wie #ZFS..."
2011-09-11,osx,47.500000,8.750000,6,"Hm, whys #OSX not using the #VPN I've just now..."
2011-05-02,chtd,47.562300,7.599410,15,I'm at Microsoft Techdays 11 Basel #chtd w/ @f...
2011-06-08,ios5,47.452575,8.278775,15,@Mirkowelle iTunes Beta installieren. #iOS5_$$...
2011-10-12,ios5,47.417550,9.094210,6,@TeleboyCH Möchte halt nicht mehr mit PC arbei...
2011-11-14,zurich,47.418000,8.558890,5,Starting to be safe to assume that the camp in...


In [32]:
print('Linked dataframe of all days:')
dictionary[new_events.iloc[0].hashtag].head(10)

Linked dataframe of all days:


,latitude,hashtag,numberOfTweets,text,longitude,event
2010-03-11,47.35320,ipad,1,Der #iPad hat bereits einen Preis gewonnen! 43...,7.907140,False
2010-03-26,47.55220,ipad,1,Dali nas http://bit.ly/cXnFst #apple zeli izne...,8.853100,False
2010-04-03,46.88080,ipad,1,Schön währs #iPad #Heute http://flic.kr/p/7Qt5ZW,8.644520,False
2010-04-05,46.96045,ipad,2,@monoblog ouu - dann wird es aber eng im bejew...,7.585460,False
2010-04-12,47.46835,ipad,2,Voll einverstanden !! Lese: http://bit.ly/duXG...,8.292905,False
2010-04-14,47.45930,ipad,1,Yeah ! Baby du fühlst dich geil an - mein iPad...,8.254480,False
2010-04-16,46.17860,ipad,3,"Allez hop, l'iPad comme vous ne l'avez encore ...",6.246840,False
2010-04-17,47.16190,ipad,1,"Uff, Instapaper sync dauert ja eeeewigs! #ipad",8.291050,False
2010-04-21,46.17860,ipad,2,Just found a real browser for #iPad : #icab f...,6.246840,False
2010-04-29,47.42790,ipad,2,@bewuethr Smells like win spirit: Nach einem N...,8.412835,False


## Exporting data

In [33]:
total_number_of_events = len(new_events)
print('There are {} events.'.format(total_number_of_events))

There are 33 events.


In [34]:
e_df = new_events.copy()
e_df['date'] = e_df.index
e_df.index = [i for i in range (len(e_df))]
e_df.head(1)

,hashtag,latitude,longitude,numberOfTweets,text,date
0,ipad,47.3157,8.55275,5,"@shibby Vielleicht ""noch"" nicht? Das #iPad mit...",2010-05-26


We are going to generate the right datetimes for the jsons:

In [35]:
epoch_dt = datetime(1970, 1, 1)
def to_utc(date):
    d_dt = datetime.combine(date, datetime.min.time())
    return int((d_dt - epoch_dt).total_seconds()*1000)

In [36]:
def convert_to_unix_time(record):
    datetime_index = pd.DatetimeIndex([datetime(record['year'], record['month'], 1)])
    unix_time_index = datetime_index.astype(np.int64) // 10**6
    return unix_time_index[0]

In [37]:
pr('Converting dates...')
e_df['year'] = e_df['date'].apply(lambda x: x.year)
e_df['month'] = e_df['date'].apply(lambda x: x.month)
e_df['utc_date'] = e_df['date'].apply(lambda x: to_utc(x))
e_df['unix_time'] = e_df.apply(convert_to_unix_time, axis=1)
pr('Done.')
e_df.head(1)

00:47:05 Converting dates...
00:47:05 Done.


,hashtag,latitude,longitude,numberOfTweets,text,date,year,month,utc_date,unix_time
0,ipad,47.3157,8.55275,5,"@shibby Vielleicht ""noch"" nicht? Das #iPad mit...",2010-05-26,2010,5,1274832000000,1272672000000


Grouping by months and generating the dictionary for the json.

In [38]:
e_gb_month = e_df.groupby(e_df.unix_time)

In [39]:
pr('Making event list...')
months = []
for month, df in e_gb_month:
    days = []
    for i in range (len(df)):
        ht = df.iloc[i]['hashtag']
        lat = df.iloc[i]['latitude']
        lon = df.iloc[i]['longitude']
        t_num = df.iloc[i]['numberOfTweets']
        tweets = df.iloc[i]['text'].split(delimiter)
        date = df.iloc[i]['utc_date']
        
        data_unit = { 'name': ht
                    , 'latitude' : lat
                    , 'longitude' : lon
                    , 'tweets' : tweets
                    , 'number_of_tweets' : str(t_num)
                    , 'date' : int(date)}
        days.append(data_unit)
    
    curr_month = {'date': int(month), 'data' : days}
    months.append(curr_month)

final_events = {'events' : months}
pr('Done.')

00:47:05 Making event list...
00:47:05 Done.


In [40]:
exportFilename = 'export_twitter_events_' + datetime.now().strftime("%Y-%m-%d_%Hh%Mmin%S") + \
'_' + str(total_number_of_events)+ '_events.json'
exportPath =  os.path.join('data', exportFilename)

pr('Exporting to json...')
with open(exportPath, 'w') as f:
     json.dump(final_events, f)
pr('Export done. File "{}" has been created.'.format(exportFilename))

00:47:05 Exporting to json...
00:47:05 Export done. File "export_twitter_events_2017-01-31_00h47min05_33_events.json" has been created.
